In [59]:
import cv2
import numpy as np 
import time
import os
from random import shuffle
from random import randint

DIVISOR = 16 #8 16 32 64


In [60]:
def viewImage(img):
    cv2.imshow("image",img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [61]:
#ranges of hsv colors that can be identified 
#[ 93 235 234] [133 275 274] close-up blue

def object_mask(hsv_img):
    keepl = np.array([93, 235, 234])
    keeph = np.array([133, 275, 274])
    keep_mask = cv2.inRange(hsv_img, keepl, keeph)
    hsv_img[keep_mask > 0] = ([255,255,255])

    return hsv_img

In [62]:
def shearing(img, y, x, h, w, background_img):
    
    
    left = img[y:y+h, x:x+int(w/2)]
    right = img[y:y+h, x+int(w/2):x+w]
    
    img[y:y+h, x:x+w] = background_img[y:y+h, x:x+w]
    
    viewImage(img)
    
    random_y = randint(0, 360)
    random_x = randint(0, 640)

    r_y = randint(0, 360)
    r_x = randint(0, 640)

    #make sure they don't leave the boundaries of the image

    if random_y + h  > 360: 
        random_y = 360 - h

    if random_x + w  > 640: 
        random_x = 640 - w

    if r_y + h  > 360: 
        r_y = 360 - h

    if r_x + w  > 640: 
        r_x = 640 - w

    img[random_y:random_y+h, random_x:random_x+int(w/2)] = left
    viewImage(img)
    img[r_y:r_y+h, r_x+int(w/2):r_x+w] = right
    viewImage(img)
    
    return img
    
    
    

In [63]:
def img_cuber(img, y, x, h, w, background_img):
    height_div = int(360 / DIVISOR)
    width_div = int(640 / DIVISOR)
    posY = 0
    posX = 0
    
    img[y:y+h, x:x+w] = background_img[y:y+h, x:x+w]
    
    cube_img = []
    
    for i in range(DIVISOR):
        for j in range(DIVISOR):
            roi = img[posY:posY+height_div, posX:posX+width_div]
            posY += height_div
            cube_img.append(roi)
            
        posX += width_div
        posY = 0
        
        
    shuffle(cube_img)

    posY = 0
    posX = 0
    
    count = 0
    
    for i in range(DIVISOR):
        for j in range(DIVISOR):
            img[posY:posY+height_div, posX:posX+width_div] = cube_img[count]
#             print(count)
            count += 1
            posY += height_div
            cube_img.append(roi)
            
        posX += width_div
        posY = 0
    
#     viewImage(img)
    


In [64]:
def main():
    

    img_depth = cv2.imread("./imgs/depth.png", cv2.IMREAD_COLOR)
    img_color = cv2.imread("./imgs/color.png", cv2.IMREAD_COLOR)
    background_img = cv2.imread("./imgs/background.jpg", cv2.IMREAD_COLOR)
    background_img = cv2.resize(background_img,(640,360))
    
    viewImage(img_depth)
    hsv_depth = cv2.cvtColor(img_depth, cv2.COLOR_BGR2HSV)

    hsv_depth = object_mask(hsv_depth)
    viewImage(hsv_depth)

    gray_depth = cv2.cvtColor(hsv_depth, cv2.COLOR_BGR2GRAY)
    ret, threshold = cv2.threshold(gray_depth, 240,255,0)
    viewImage(threshold)

    contours, hierarchy = cv2.findContours(threshold,cv2.RETR_TREE,cv2.CHAIN_APPROX_NONE)

    for c in contours:
        
        area = cv2.contourArea(c)
        
        if area > 2000:
        
            x, y, w, h = cv2.boundingRect(c)
            roi = img_color[y:y+h, x:x+w]
            img2=img_color.copy()
            img3=img_color.copy()
            img_cuber(img2, y, x, h, w, background_img)
            img2[y:y+h, x:x+w]=roi
            background_img[y:y+h, x:x+w]=roi
            img3 = shearing(img3, y, x, h, w, background_img)
            cv2.rectangle(img_color,(x,y),(x+w,y+h),(0,255,0),3)
    #cv2.drawContours(img_color, contours, -1, (0,255,0), 3)
#     viewImage(img_color)
    viewImage(img2)
    viewImage(img3)
    viewImage(background_img)
    
    



In [65]:
main()